### Python documentation for the MIDS program
####Due November 20 2015 to Courtney

What I want to do:
- Import all data from actigraphs, azumio and fitbit
- Make graphs to compare all for each user
- With all data individually:
    Average steps over a 7 day period
- With actigraph data, get MVPA

In [110]:
import pandas as pd
import numpy as np

In [132]:
accel = pandas.read_csv('/Users/megan/Desktop/Py project/Accel CSV data/User701_ActigraphMay.csv', skipinitialspace=True)

In [133]:
# Limiting dates to the days of the study
date1 = np.array(accel.Date > "5/16/2015")
date2 = np.array(accel.Date < "5/25/2015")
wear_dates = np.logical_and(date1, date2)

accel = accel[wear_dates]

In [134]:
# Grouping by study day and finding out total steps per day
by_date = accel.groupby("Date")
daily_steps = by_date['Steps'].sum()

In [135]:
# Defining minutes of moderate and vigorous physical activity
mvpa = np.array(accel.Axis1 > 1951)

# Creating a dataframe with only minutes of mvpa included
accel_mvpa = accel[mvpa]

# Getting total minutes of MVPA for each day
by_date_mvpa = accel_mvpa.groupby("Date")
daily_mvpa = by_date_mvpa['Axis1'].count()


In [136]:
# Creating a dataframe that includes daily steps and daily minutes of MVPA for this user
daily_701 = pd.DataFrame({'User': 701, 'Steps': daily_steps, 'MVPA' : daily_mvpa}).reset_index()

In [137]:
daily_701

,Date,MVPA,Steps,User
0,5/17/2015,20,5851,701
1,5/18/2015,66,9656,701
2,5/19/2015,41,6649,701
3,5/20/2015,56,7936,701
4,5/21/2015,51,8030,701
5,5/22/2015,75,11172,701
6,5/23/2015,52,8046,701
7,5/24/2015,60,8624,701
